In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam


In [3]:
column_names = ['Image', 'Attention']

df = pd.read_csv('attention.csv', header=None, names=column_names)
df['Image'] = 'resized_images/' + df['Image']

# Tambahkan path gambar
# df['filepath'] = df[''].apply(lambda x: os.path.join("resized_images", x))
df.head()


,Image,Attention
0,resized_images/1.png,1
1,resized_images/2.png,2
2,resized_images/3.png,1
3,resized_images/4.png,4
4,resized_images/5.png,1


In [5]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['Attention'], random_state=42)


In [8]:
IMG_SIZE = (600, 600)

def load_images(df):
    images = []
    labels = []
    for _, row in df.iterrows():
        img = load_img(row['Image'], target_size=IMG_SIZE)
        img = img_to_array(img)
        img = preprocess_input(img)
        images.append(img)
        labels.append(row['Attention'])
    return np.array(images), tf.keras.utils.to_categorical(labels, num_classes=5)

X_train, y_train = load_images(train_df)
X_val, y_val = load_images(val_df)


In [9]:
base_model = EfficientNetB7(include_top=False, input_shape=(600, 600, 3), weights='imagenet')
base_model.trainable = False  # Transfer Learning (freeze dulu)

inputs = Input(shape=(600, 600, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
outputs = Dense(5, activation='softmax')(x)
model = Model(inputs, outputs)

model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 107s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 600, 600, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb7 (Functional)     │ (None, 19, 19, 2560)   │    64,097,687 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2560)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │        12,805 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,110,492 (244.56 MB)

 Trainable params: 12,805 (50.02 KB)

 Non-trainable params: 64,097,687 (244.51 MB)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=16
)


Epoch 1/10
14/15 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step - accuracy: 0.2712 - loss: 1.5670

In [ ]:
# Evaluasi
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc:.4f}")

# Prediksi
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)

# Classification report
print(classification_report(y_true, y_pred_classes))

# Plot akurasi
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')
plt.show()
